In [64]:
%pwd
%cd /Users/ashwin/Desktop/College Work/Fourth Year/L48/GroupProject/tti

/Users/ashwin/Desktop/College Work/Fourth Year/L48/GroupProject/tti


In [65]:
import os

import numpy as np
import pandas as pd
from tqdm.notebook import trange
from tqdm import tqdm
import importlib

import tti_explorer
from tti_explorer import config, utils
from tti_explorer.case import simulate_case, CaseFactors
from tti_explorer.contacts import EmpiricalContactsSimulator
from tti_explorer.strategies import TTIFlowModel, RETURN_KEYS

importlib.reload(tti_explorer)
importlib.reload(tti_explorer.config)
importlib.reload(tti_explorer.utils)
importlib.reload(tti_explorer.case)
importlib.reload(tti_explorer.strategies)
importlib.reload(tti_explorer.contacts)


<module 'tti_explorer.contacts' from '/Users/ashwin/Desktop/College Work/Fourth Year/L48/GroupProject/tti/tti_explorer/contacts.py'>

In [66]:
def print_doc(func):
    print(func.__doc__)

In [67]:
name = 'S3_test_based_TTI'
rng = np.random.RandomState(0)
case_config = config.get_case_config("delve_mostly_uni")
#case_config = config.get_case_config("delve_mostly_old")
#case_config = config.get_case_config("delve_mostly_twenties")
#contacts_config = config.get_contacts_config("delve")
print(contacts_config)
policy_config = config.get_strategy_configs("delve", name)[name]
factor_config = utils.get_sub_dictionary(policy_config, config.DELVE_CASE_FACTOR_KEYS)
strategy_config = utils.get_sub_dictionary(policy_config, config.DELVE_STRATEGY_FACTOR_KEYS)

{'home_sar': 0.3, 'work_sar': 0.045, 'other_sar': 0.045, 'period': 10, 'asymp_factor': 0.5}


In [68]:
def load_csv(pth):
    return np.loadtxt(pth, dtype=int, skiprows=1, delimiter=",")

In [69]:
path_to_bbc_data = os.path.join("data", "processed")

child_no_school = load_csv(os.path.join(path_to_bbc_data, "childnoschool.csv"))
child_school = load_csv(os.path.join(path_to_bbc_data, "childschool.csv"))
university = load_csv(os.path.join(path_to_bbc_data, "university.csv"))
twenties = load_csv(os.path.join(path_to_bbc_data, "twenties.csv"))
thirties_to_fifties = load_csv(os.path.join(path_to_bbc_data, "thirtiestofifties.csv"))
fifties_to_seventies = load_csv(os.path.join(path_to_bbc_data, "fiftiestoseventies.csv"))
seventy_plus = load_csv(os.path.join(path_to_bbc_data, "seventyplus.csv"))

In [70]:
simulate_contacts = EmpiricalContactsSimulator(child_no_school, child_school, university, twenties, thirties_to_fifties, fifties_to_seventies, seventy_plus, rng)
tti_model = TTIFlowModel(rng, **strategy_config)

In [71]:
n_seperate_starting_cases = 10000
n_repetitions = 10
outputs = list()
death_rates = []

for i in tqdm(range(n_seperate_starting_cases)):
    old_probs = []
    for i in range(n_repetitions):
        case = simulate_case(rng, p_for_categories_continued=old_probs, **case_config)
        case_factors = CaseFactors.simulate_from(rng, case, **factor_config)
        (contacts, old_probs, death_rate) = simulate_contacts(case, **contacts_config)
        res = tti_model(case, contacts, case_factors)
        outputs.append(res)
        death_rates.append(death_rate)

100%|██████████| 10000/10000 [03:19<00:00, 50.05it/s]


In [72]:
# This cell is mosltly just formatting results...
to_show = [
    RETURN_KEYS.base_r,
    RETURN_KEYS.reduced_r,
    RETURN_KEYS.man_trace,
    RETURN_KEYS.app_trace,
    RETURN_KEYS.tests
]

# scale factor to turn simulation numbers into UK population numbers
nppl = case_config['infection_proportions']['nppl']
scales = [1, 1, nppl, nppl, nppl]

results = pd.DataFrame(
    outputs
).mean(
    0
).loc[
    to_show
].mul(
    scales
).to_frame(
    name=f"Simulation results: {name.replace('_', ' ')}"
).rename(
    index=lambda x: x + " (k per day)" if x.startswith("#") else x
)

results.round(3)

,Simulation results: S3 test based TTI
Base R,4.061
Effective R,1.356
# Manual Traces (k per day),107.618
# App Traces (k per day),15.380
# Tests Needed (k per day),89.509


In [73]:
relevant_death_rate_sum = sum(n for n in death_rates if n != -1)
relevant_death_rate_count = sum(1 for n in death_rates if n != -1)
dr = relevant_death_rate_sum / relevant_death_rate_count
print("Average Death Rate: " + str(round(dr*100,2)) + "%")

Average Death Rate: 1.24%
